In [71]:
import numpy as np

from scipy import ndimage

https://adventofcode.com/2020

# Day 17

In [5]:
input17 = """..##.#.#
.#####..
#.....##
##.##.#.
..#...#.
.#..##..
.#...#.#
#..##.##"""

In [51]:
def map_to_array(astmap):
    arr2d = np.array([[e=='#' for e in row] for row in astmap.split('\n')]).T
    return arr2d.copy()

def array_to_map(arr, marker='#'):
    buffer = []
    for row in arr.T:
        for element in row:
            buffer.append(marker if element else '.')
        buffer.append('\n')
    return ''.join(buffer)

## Part 1

In [53]:
test_input = """.#.
..#
###"""
testarr = map_to_array(test_input)
print(array_to_map(testarr))

.#.
..#
###



In [65]:
boot_field = np.zeros((testarr.shape[0] + 6*2, testarr.shape[1] + 6*2, 1 + 6*2), dtype=bool)
boot_field[6:9, 6:9, 6] = testarr
print(array_to_map(boot_field[..., 6]))

...............
...............
...............
...............
...............
...............
.......#.......
........#......
......###......
...............
...............
...............
...............
...............
...............



In [101]:
neighbor_kernel = [[[1,1,1],
                    [1,1,1],
                    [1,1,1]],
                   [[1,1,1],
                    [1,0,1],
                    [1,1,1]],
                   [[1,1,1],
                    [1,1,1],
                    [1,1,1]]]
neighbor_kernel = np.array(neighbor_kernel)
assert np.sum(neighbor_kernel) == 26

def do_step(initial_field, nsteps=1):
    if nsteps==0:
        return initial_field.reshape(initial_field.shape[0], initial_field.shape[1], 1)
    
    boot_field = np.zeros((initial_field.shape[0] + nsteps*2, 
                           initial_field.shape[1] + nsteps*2, 
                           1 + nsteps*2), dtype=int)
    boot_field[nsteps:(nsteps + initial_field.shape[0]), 
               nsteps:(nsteps + initial_field.shape[1]), 
               nsteps] = initial_field.astype(int)
    
    for i in range(nsteps):
        corr = ndimage.correlate(boot_field, neighbor_kernel, mode='constant')
        new_field = boot_field & (corr==2)
        new_field[corr==3] = True
        boot_field = new_field.astype(int)
    
    return boot_field

step3 = do_step(testarr, nsteps=3)
                     
                     
assert array_to_map(step3[1:-1, 2:, 3]) == """...#...
.......
#......
.......
.....##
.##.#..
...#...
"""

assert np.sum(do_step(testarr, nsteps=6)) == 112

In [102]:
np.sum(do_step(map_to_array(input17), nsteps=6))

213

## Part 2 

In [105]:
neighbor_kernel4 = np.ones((3, 3, 3, 3), dtype=int)
neighbor_kernel4[1,1,1,1] = 0
neighbor_kernel4 = np.array(neighbor_kernel4)
assert np.sum(neighbor_kernel4) == 80

In [117]:
def do_step4(initial_field, nsteps=1):
    if nsteps==0:
        return initial_field.reshape(initial_field.shape[0], initial_field.shape[1], 1, 1)
    
    boot_field = np.zeros((initial_field.shape[0] + nsteps*2, 
                           initial_field.shape[1] + nsteps*2, 
                           1 + nsteps*2, 1 + nsteps*2), dtype=int)
    boot_field[nsteps:(nsteps + initial_field.shape[0]), 
               nsteps:(nsteps + initial_field.shape[1]), 
               nsteps, nsteps] = initial_field.astype(int)
    
    for i in range(nsteps):
        corr = ndimage.correlate(boot_field, neighbor_kernel4, mode='constant')
        new_field = boot_field & (corr==2)
        new_field[corr==3] = True
        boot_field = new_field.astype(int)
    
    return boot_field

assert array_to_map(do_step4(testarr, 2)[1:-1, 2:, 2, 0]) == """###..
##.##
#...#
.#..#
.###.
""""""

In [131]:
assert np.sum(do_step4(testarr, nsteps=6)) == 848

In [132]:
np.sum(do_step4(map_to_array(input17), nsteps=6))

1624

# Day 18

In [3]:
with open('input18') as f:
    input18 = f.read().strip()

## Part 1

For loop over lines is probably ok for modest-size inputs

In [101]:
def find_subexpressions(s):
    sub_exprs = {}
    if '(' in s:
        paren_level = 0
        for i, si in enumerate(s):
            if si=='(':
                paren_level += 1
                if paren_level == 1:
                    sub_exprs['current'] = i
            elif si == ')':
                paren_level -= 1
                if paren_level == 0:
                    sub_exprs[sub_exprs['current']] = i
                    del sub_exprs['current']
        if paren_level != 0:
            raise ValueError('parens not matched!')
    return sub_exprs
    

def evaluate_expression(s):
    sub_exprs = find_subexpressions(s)
    # accumulate
    result = 0
    
    operator = 'start'
    nums = tuple('1234567890')
    i = 0
    while i < len(s):
        si = s[i]
        
        if i in sub_exprs:
            substr = s[i+1:sub_exprs[i]]
            subres = evaluate_expression(substr)
            if operator == 'start':
                result = subres
            elif operator is None:
                raise ValueError('syntax error - num does not follow operator')
            elif operator == '+':
                result += subres
            elif operator == '*':
                result *= subres
            operator = None
            j = sub_exprs[i]+2
            i = j
            continue
        elif si in ('*', '+'):
            operator = si
        elif si in nums:
            if operator == 'start':
                result += int(si)
            elif operator is None:
                raise ValueError('syntax error - num does not follow operator')
            elif operator == '+':
                result += int(si)
            elif operator == '*':
                result *= int(si)
                operator = None
        elif si == ' ':
            pass
        else:
            raise ValueError(f"don't know what to do with character {si}")
        i += 1
    return result

assert evaluate_expression('9 + 3 * 4') == (9 + 3) * 4
assert evaluate_expression('9 + (3 * 4)') == 9 + 3 * 4 
assert evaluate_expression('1 + (3 + 2) + ((1 + 2) + 3)') == 1 + (3 + 2) + ((1 + 2) + 3)

In [102]:
%timeit evaluate_expression('1 + (3 + 2) + ((1 + 2) + 3)')

9.38 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [103]:
sum([evaluate_expression(line) for line in input18.split('\n')])

7147789965219

## Part 1b

As feared, the above doesn't really work with part 2...

In [148]:
NUMS = tuple('1234567890')
OPERATORS = ('*', '+')

class Number:
    def __init__(self, num):
        self.num = int(num)
    def __call__(self):
        return self.num
    
class Expression:
    def __init__(self, s):
        self.expressions = []
        self.operators = []
        
        sub_exprs = find_subexpressions(s)
        
        i = 0
        while i < len(s):
            si = s[i]
            if si in NUMS:
                self.expressions.append(Number(si))
            if si in OPERATORS:
                self.operators.append(si)
            elif i in sub_exprs:
                self.expressions.append(self.__class__(s[i+1:sub_exprs[i]]))
                j = sub_exprs[i]+2
                i = j
                continue
            i += 1
        if len(self.expressions)-1!= len(self.operators):
            raise ValueError(f'operators and expressions not matched! expressions:{self.expressions}, operators:{self.operators}')
    def __call__(self):
        evaled = [e() for e in self.expressions]
        result = evaled[0]
        for i, op in enumerate(self.operators):
            if op == '+':
                result += evaled[i+1]
            elif op == '*':
                result *= evaled[i+1]
            else:
                raise ValueError(f'Invalid operator "{op}"')
        return result
            
assert Expression('2 * 3 + (4 * 5)')() == 26
assert Expression('5 + (8 * 3 + 9 + 3 * 4 * 3)')() == 437
assert Expression('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')() == 12240
assert Expression('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2')() == 13632

In [150]:
assert sum([Expression(line)() for line in input18.split('\n')]) == 7147789965219

## Part 2 

In [169]:
class PlusPrecedenceExpression(Expression):
    def __call__(self):
        evaled = [e() for e in self.expressions]
        ops = self.operators.copy()
        result = evaled[0]
        for i in range(len(self.operators))[::-1]:
            op = ops[i]
            if op == '+':
                result = evaled[i] + evaled[i+1]
                del evaled[i+1]
                evaled[i] = result
                del ops[i]
            elif op == '*':
                pass
            else:
                raise ValueError(f'Invalid operator "{op}"')
        return np.prod(evaled)
    
assert PlusPrecedenceExpression('1 + (2 * 3) + (4 * (5 + 6))')() == 51
assert PlusPrecedenceExpression('2 * 3 + (4 * 5)')() == 46
assert PlusPrecedenceExpression('5 + (8 * 3 + 9 + 3 * 4 * 3)')() == 1445
assert PlusPrecedenceExpression('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')() == 669060
assert PlusPrecedenceExpression('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2')() == 23340

In [170]:
sum([PlusPrecedenceExpression(line)() for line in input18.split('\n')])

136824720421264

# Day 19

In [2]:
with open('input19') as f:
    input19 = f.read().strip()

## Part 1

In [55]:
basic_test_input = '''0: 1 2
1: "a"
2: 1 3 | 3 1
3: "b"'''

def parse_input(s):
    rules = {}
    messages = []
    in_rules = True
    for l in s.strip().split('\n'):
        if l.strip() == '':
            in_rules = False
        elif in_rules:
            code, rule = l.split(':')
            if '"' in rule:
                rules[int(code)] = rule.replace('"', '').strip()
            else:
                rules[int(code)] = [tuple([int(n) for n in grp.split()]) for grp in rule.split('|')]
        else:
            messages.append(l.strip())
    return rules, messages
            
basic_rules, messages = parse_input(basic_test_input)  
basic_rules, messages

({0: [(1, 2)], 1: 'a', 2: [(1, 3), (3, 1)], 3: 'b'}, [])

In [94]:
def check_message(message, rules, start_index):
    my_rules = rules[start_index]
    isstring = isinstance(my_rules, str)
        
    if isinstance(my_rules, str):
        if message[0] == my_rules:
            return message[1:]
        else:
            return False
    else:
        for ruleset_or in my_rules:
            current_message = message
            for rule in ruleset_or:
                if current_message == '' or current_message is True:
                    return False
                current_message = check_message(current_message, rules, rule)
                if current_message is False:
                    break
            else:
                if current_message == '':
                    return True
                else:
                    return current_message
        else:
            return False
        
check_message('aab', basic_rules, 0), check_message('aba', basic_rules, 0), check_message('abb', basic_rules, 0)

(True, True, False)

In [95]:
test_input = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""
rules, messages = parse_input(test_input)
rules, messages

({0: [(4, 1, 5)],
  1: [(2, 3), (3, 2)],
  2: [(4, 4), (5, 5)],
  3: [(4, 5), (5, 4)],
  4: 'a',
  5: 'b'},
 ['ababbb', 'bababa', 'abbbab', 'aaabbb', 'aaaabbb'])

In [96]:
assert np.all(np.array([check_message(message, rules, 0) is True for message in messages]) == 
              [True, False, True, False, False])

In [97]:
rules, messages = parse_input(input19)
sum([check_message(message, rules, 0) is True for message in messages])

156

## Part 2 

In [98]:
updated_input19 = input19.replace('8: 42', '8: 42 | 42 8').replace('11: 42 31', '11: 42 31 | 42 11 31')

In [99]:
rules, messages = parse_input(updated_input19)
sum([check_message(message, rules, 0) is True for message in messages])

199

Hmm, it's not doing infinite recursion.  Clearly an algorithm flaw

# Day N

In [ ]:
!mv ~/Desktop/input.txt inputN

In [ ]:
with open('inputN') as f:
    inputN = f.read().strip()

## Part 1

## Part 2 